## <span style="color:green"> Notebook - Phase mixing functions </span>
<span style="color:purple">Sinan Ozaydin, School of Geosciences, The University of Sydney, NSW 2006, Australia <br/> </span>
<span style="color:purple">sinan.ozaydin@protonmail.com</span>

This notebook will demonstrate how to use the phase mixing functions for electrical conductivity models. Phase mixing functions are used to calculate the bulk conductivity of a mineral assemblage with the given geometrical assumptions. Since electrical conductivity is a measure of how easily can electrons can flow through a system, the connectivity of conductive phases in a matrix has crucial importance when it comes to interpreting the electrical conductivity variations measured with MT method.

In ***pide***, there are six solid-state phase mixing functions are available to use. Which can be listed with the code snippet below:


In [1]:
import pide
import numpy as np
import matplotlib.pyplot as plt

p_obj = pide.pide() #forming the pide object
p_obj.list_phs_mix_methods()

Solid Phases Mixing Models:
0.   Generalized Archie's Law (Glover, 2010)
1.   Hashin-Shtrikman Lower Bound (Berryman, 1995)
2.   Hashin-Shtrikman Upper Bound (Berryman, 1995)
3.   Parallel Model (Guegen and Palciauskas, 1994)
4.   Perpendicular Model (Guegen and Palciauskas, 1994)
5.   Random Model (Guegen and Palciauskas, 1994)


["Generalized Archie's Law (Glover, 2010)",
 'Hashin-Shtrikman Lower Bound (Berryman, 1995)',
 'Hashin-Shtrikman Upper Bound (Berryman, 1995)',
 'Parallel Model (Guegen and Palciauskas, 1994)',
 'Perpendicular Model (Guegen and Palciauskas, 1994)',
 'Random Model (Guegen and Palciauskas, 1994)']

$$\sigma = \sum_{i = 1}^n \sigma_{i} \phi^{m} \qquad \text{(1)} \quad \text{Generalized Archie's Law}$$
$$\sigma = \left( \sum_{i = 1}^n \frac{\phi_i}{\sigma_i + 2\sigma_{min}}   \right)^{-1} - 2\sigma_{min} \qquad \text{(2)} \quad \text{Hashin-Shtrikman Lower Bound}$$
$$\sigma = \left( \sum_{i = 1}^n \frac{\phi_i}{\sigma_i + 2\sigma_{max}}   \right)^{-1} - 2\sigma_{max} \qquad \text{(3)} \quad \text{Hashin-Shtrikman Upper Bound}$$
$$\sigma = \sum_{i = 1}^n \phi_i \sigma_i \quad \text{(4)} \quad \text{Parallel Model}$$
$$\sigma = \sum_{i = 1}^n \frac{\sigma_i}{\phi_i} \quad \text{(5)} \quad \text{Perpendicular Model}$$
$$\sigma = \prod \sigma_i^{\phi_i} \quad \text{(6)} \quad \text{Random Model}$$


where volumetrically dominant cementation exponent for m in Generalized Archie's law is:

$$m_j = log \left( 1 - \sum_{i \neq j} \phi_i^{m_j} \right) / log \left( 1 - \sum_{i \neq j} \phi_i \right) \quad \text{(7)}$$


In [2]:
temp = np.arange(600,1300,5) #setting up temperature array
p_obj = pide.pide() #creating the initial object

p_obj.set_temperature(temp) #setting temperature array in K
p_obj.set_pressure(1.0) #GPa

#Setting a basic garnet-lherzolite matrix
p_obj.set_composition_solid_mineral(ol = 0.6,opx = 0.25,cpx = 0.1, garnet= 0.05)
#Setting m values for Generalised Archie's Law
#Here the m value for the dominant phase will be overwritten and will be recalculated as m_j
p_obj.set_phase_interconnectivities(ol = 1, opx = 2, cpx = 4, gt = 4, melt = 1)

In [3]:
#Setting bulk water(hydroxyl(OH^-1)) content to 100 ppm H2O wt
p_obj.set_bulk_water(100)
#Now we have to distribute this water content using water partitioning coefficients.
#To do this, let's list all the possible water partitioning coefficients first, for the minerals we used.
p_obj.list_mantle_water_partitions_solid('opx')

Mantle solid-state water partition coefficients for the mineral: opx
0.  Ozaydin2020_Opx1 -  Type 1
1.  Ozaydin2020_Opx2 -  Type 1
2.  Aubaud2004_Opx -  Type 0  -  Opx/Ol: 8.92
3.  Demouchy2017_Opx -  Type 0  -  Opx/Ol: 5.6
4.  Ferot2012_Opx -  Type 1
5.  Grant2006_Opx -  Type 0  -  Opx/Ol: 2.52
6.  Hauri2006_Opx -  Type 0  -  Opx/Ol: 6.93
7.  Koga2003_Opx -  Type 0  -  Opx/Ol: 12.0
8.  Kovacs2012_Opx -  Type 0  -  Opx/Ol: 5.34
9.  Kovacs2012_Opx2 -  Type 1
10.  Novella2014_Opx -  Type 0  -  Opx/Ol: 1.91
11.  Sakurai2014_Opx -  Type 1
12.  Withers2011_Opx1 -  Type 0  -  Opx/Ol: 1.505
13.  Withers2011_Opx2 -  Type 1
                 
                 
